In [4]:
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim

class NetWork(nn.Module):
    def __init__(self, input_feature, output):
        super().__init__()
        self.fc1 = nn.Linear(input_feature, output, bias=False)
        nn.init.xavier_normal_(self.fc1.weight)  # 重みをXavierの初期化
        self.fc2 = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        return x

# 加载数据
X_train = torch.load("X_train.pt")
Y_train = torch.load("Y_train.pt")

# 确认是否有可用的GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 将数据移到设备上
X_train = X_train.to(device)
Y_train = Y_train.to(device)

# 定义损失函数和优化器
loss = nn.CrossEntropyLoss(reduction="mean")
model = NetWork(300, 4).to(device)
ds = TensorDataset(X_train, Y_train)
DL = DataLoader(ds, batch_size=256, shuffle=True)
optimizer = optim.SGD(params=model.parameters(), lr=0.01)

# 训练模型
epoch = 100
for ep in range(epoch):
    for X, Y in DL:
        X = X.to(device)
        Y = Y.to(device)
        Y_pred = model(X)
        CEloss = loss(Y_pred, Y)
        optimizer.zero_grad()
        CEloss.backward()
        optimizer.step()

# 保存模型
torch.save(model.state_dict(), "SingleLayer_CPU.pth")
